In [ ]:
include("../finite_difference.jl")
include("../ode_solver.jl")

kappa = 1   # diffusion constant
L = 1.0         # length of spatial domain
T = 0.5         # total time to solve for

function u_I(x)
    # initial temperature distribution
    y = sin.(pi*x/L)
    return y
end

function u_exact(x,t)
    # the exact solution
    y = exp.(-kappa*(pi^2/L^2)*t)*sin.(pi*x/L)
    return y
end

# Set numerical parameters
mx = 10     # number of gridpoints in space
mt = 1000   # number of gridpoints in time


# forward euler Estimate
weird_x, u_j = finite_difference(u_I, kappa, L, T, mx, mt, forward_euler)
# Create trace
f_euler = scatter(x=x, y=u_j, mode="markers", name="forward euler", showlegend=true)


# backward euler Estimate
x, u_j = finite_difference(u_I, kappa, L, T, mx, mt, backward_euler)
# Create trace
b_euler = scatter(x=x, y=u_j, mode="markers", name="backward euler", showlegend=true)


# crank nicholson Estimate
x, u_j = finite_difference(u_I, kappa, L, T, mx, mt, crank_nicholson)
# Create trace
c_nicholson = scatter(x=x, y=u_j, mode="markers", name="crank nicholson", showlegend=true)



# Plot the final result and exact solution
xx = LinRange(0,L,250)

# Create solution trace
exact = scatter(x=xx, y=u_exact(xx,T), mode="lines", name="exact", showlegend=true)

layout = Layout(
    xaxis_title = "x",
    yaxis_title = "u(x,0.5)"
    )

plot([exact, f_euler, b_euler, c_nicholson], layout)

In [33]:
using PlotlyJS
using Test
include("../ode_solver.jl")
include("../numerical_shooting.jl")
include("../examples/example_functions.jl")
include("../visualisation.jl")
include("../finite_difference.jl")

crank_nicholson (generic function with 1 method)

In [53]:


@testset verbose = true "Systems Tests" begin

    @testset verbose = true "Week 3 - ode_solver" begin

        @testset verbose = true "Input Tests" begin

            # test error is thrown if t=0 is not included in t
            t = 0:0.1:1
            bad_t = 1:0.1:2
            @test_throws ErrorException solve_ode(f2, [1], bad_t, "rk4")


            # test error is thrown if x0 is not a matrix
            @test_throws ErrorException solve_ode(f2, 1, t, "rk4")
            @test_throws ErrorException solve_ode(f2, [1;0], t, "rk4")
            @test_throws ErrorException solve_ode(f2, [1, 0], t, "rk4")

            # test error is thrown if x0 is not correct length
            @test_throws ErrorException solve_ode(f2, [1], t, "rk4")

        end

        @testset verbose = true "Output Tests" begin

            # test if solve_ode estimates a simple ODE correctly
            x0 = [1]
            t = 0:1
            e_estimate = solve_ode(f, x0, t, "rk4")[end][1]
            @test isapprox(e_estimate, ℯ)


            # test if solve_ode estimates systems of ODEs correctly
            x0 = [1 0]
            t = 0:0.1:1
            f2_sol = f2_solution(x0, t)
            f2_numerical_sol = solve_ode(f2, x0, t, "rk4")
            @test  all(isapprox.(f2_numerical_sol, f2_sol, atol=1e-6))

        end
            

    end

    @testset verbose = true "Week 17 - Hopf numerical_shooting" begin

        # test if found limit cycle matches the analytical solution
        u0, T = find_limit_cycle(hopf2d, [-1 0], 6)
        @test isapprox(T, 2*pi)


        # test if solve_ode estimates a Hopf ODE correctly
        t = 0:0.1:T
        hopf_solution = hopf2d_sol(t, beta=1, theta=pi) # adjusted for phase
        hopf_numerical_sol = solve_ode(hopf2d, u0, t, "rk4")
        @test  all(isapprox.(hopf_numerical_sol, hopf_solution, atol=1e-6))





    end
end

Test Summary:                       | Pass  Total
Systems Tests                       |    9      9
  Week 3 - ode_solver               |    7      7
    Input Tests                     |    5      5
    Output Tests                    |    2      2
  Week 17 - Hopf numerical_shooting |    2      2


Test.DefaultTestSet("Systems Tests", Any[Test.DefaultTestSet("Week 3 - ode_solver", Any[Test.DefaultTestSet("Input Tests", Any[], 5, false, true), Test.DefaultTestSet("Output Tests", Any[], 2, false, true)], 0, false, true), Test.DefaultTestSet("Week 17 - Hopf numerical_shooting", Any[], 2, false, true)], 0, false, true)

Test Summary: | Pass  Total
Systems Tests |    8      8
